# Generate daily VWAP using dedicated service - F#

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** `daily` VWAP price.

### Inputs/outputs
Daily VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the daily VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows how to plot a simple daily vwap for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve VWAP indicator from the hosted service. The queried endpoint in this script are:
* *DailyVwapsService*: to directly retrieve daily VWAP objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily VWAP sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Daily.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily vwap* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyVwapsService* in order to retrieve daily data

In [ ]:
// Set instrument
let ticker = "ACA"
let exchange = "XPAR"

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily vwap service
let service = new DailyVwapsService.DailyVwapsServiceClient(channel)

// Create and process daily vwap request
let request = 
    new DailyVwapsRequest(
        Identifier = new Identifier(Exchange = exchange, Ticker = ticker)
    )

let reply = service.DailyVwaps(request, headers)

The following code snippet displays the *daily vwap* data points:

In [ ]:
reply.Data
|> display

### Step 4: Visualize data

#### 4.1 Plot vwap and volume
The following code snippets retrive *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Extract the daily vwap and the volume
let vwaps = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Price))
let volumes = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume))

In [ ]:
// Create the graph
let vwapGraph = 
    [
        new Scattergl(
            name = "Vwap",
            mode = "lines",
            x = (vwaps |> Seq.map fst),
            y = (vwaps |> Seq.map snd)
        )
        new Scattergl(
            name = "Volume",
            mode = "bar",
            x = (volumes |> Seq.map fst),
            y = (volumes |> Seq.map snd),
            yaxis = "y2"
        )
    ]


let layout = 
    new Layout.Layout(
        title = $"Vwap & Volume for {ticker}",
        yaxis = new Yaxis (
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Volume", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(vwapGraph, layout)
|> Chart.WithXTitle "Time"
|> Chart.WithWidth  1500
|> Chart.WithHeight 800
|> display